In [12]:
import pynumdiff
import pynumdiff.optimize
import os
import numpy as np
import pandas as pd

In [18]:
def ppsd_profiles(base_dir, suite_name):
    # Define input/output directories
    density_dir = os.path.join(base_dir, "output", suite_name, "density_profiles")
    mass_dir = os.path.join(base_dir, "output", suite_name, "mass_profiles")
    velocity_dir = os.path.join(base_dir, "output", suite_name, "velocity_profiles")
    output_dir = os.path.join(base_dir, "output", suite_name, "ppsd_profiles")
    os.makedirs(output_dir, exist_ok=True)

    # Collect file names
    density_files = sorted([f for f in os.listdir(density_dir) if f.endswith(".csv")])
    mass_files = sorted([f for f in os.listdir(mass_dir) if f.endswith(".csv")])
    velocity_files = sorted([f for f in os.listdir(velocity_dir) if f.endswith(".csv")])

    # Loop through halos and compute Q_r and Q_tot
    for halo_idx, (f_rho, f_mass, f_vel) in enumerate(zip(density_files, mass_files, velocity_files)):
        df_rho = pd.read_csv(os.path.join(density_dir, f_rho))
        df_mass = pd.read_csv(os.path.join(mass_dir, f_mass))
        df_vel = pd.read_csv(os.path.join(velocity_dir, f_vel))

        # Load profile data
        r = df_rho["r_scaled"].values
        rho = df_rho["rho_scaled"].values
        m = df_mass["m_scaled"].values
        sigma_rad = df_vel["sigma_rad_scaled"].values
        sigma_tot = df_vel["sigma_total_scaled"].values

        # Compute pseudo phase-space densities
        with np.errstate(divide="ignore", invalid="ignore"):
            Q_r = np.where(sigma_rad > 0, rho / sigma_rad**3, np.nan)
            Q_tot = np.where(sigma_tot > 0, rho / sigma_tot**3, np.nan)

        # Save to CSV
        df_out = pd.DataFrame({
            "r_scaled": r,
            "m_scaled": m,
            "Q_r": Q_r,
            "Q_tot": Q_tot
        })
        df_out.to_csv(f"{base_dir}/output/{suite_name}/ppsd_profiles/halo_{halo_idx:03d}_profile.csv", index=False)

    print(f"[Saved] PPSD profiles for {suite_name} saved to {output_dir}")


In [19]:
base_dir = "/Users/fengbocheng/Projects/Symphony-PPSD"
suite_names = [
    "SymphonyLMC",
    "SymphonyMilkyWay",
    "SymphonyGroup",
    "SymphonyLCluster",
]

for suite in suite_names:
    ppsd_profiles(base_dir, suite)

[Saved] PPSD profiles for SymphonyLMC saved to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyLMC/ppsd_profiles
[Saved] PPSD profiles for SymphonyMilkyWay saved to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyMilkyWay/ppsd_profiles
[Saved] PPSD profiles for SymphonyGroup saved to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyGroup/ppsd_profiles
[Saved] PPSD profiles for SymphonyLCluster saved to /Users/fengbocheng/Projects/Symphony-PPSD/output/SymphonyLCluster/ppsd_profiles


In [20]:
# NOTE: this is the version that smoothing the derivative of density and velocity sperately and then combine them to get the smoothing derivative of ppsd
def get_diff_and_optimize_funcs(method):
    submodules = [
        'kalman_smooth',
        'smooth_finite_difference',
        'finite_difference',
        'total_variation_regularization',
        'linear_model'
    ]
    for submod in submodules:
        try:
            mod_optimize = getattr(pynumdiff.optimize, submod)
            mod_diff = getattr(pynumdiff, submod)
            if hasattr(mod_optimize, method) and hasattr(mod_diff, method):
                return getattr(mod_diff, method), getattr(mod_optimize, method)
        except AttributeError:
            continue
    raise ValueError(f"Method '{method}' not found in any submodule.")

def fit_and_save_ppsd_slopes(base_dir, suite_name, method='constant_jerk', tvgamma=None):
    density_dir = os.path.join(base_dir, "output", suite_name, "density_profiles")
    velocity_dir = os.path.join(base_dir, "output", suite_name, "velocity_profiles")
    mass_dir = os.path.join(base_dir, "output", suite_name, "mass_profiles")
    slope_r_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_r")
    slope_m_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_m")
    os.makedirs(slope_r_dir, exist_ok=True)
    os.makedirs(slope_m_dir, exist_ok=True)

    density_files = sorted([f for f in os.listdir(density_dir) if f.endswith(".csv")])
    velocity_files = sorted([f for f in os.listdir(velocity_dir) if f.endswith(".csv")])
    mass_files = sorted([f for f in os.listdir(mass_dir) if f.endswith(".csv")])
    n_halos = len(density_files)

    def fit_derivative(y, dt):
        try:
            diff_func, optimize_func = get_diff_and_optimize_funcs(method)
            kwargs = {'tvgamma': tvgamma} if 'tvgamma' in optimize_func.__code__.co_varnames else {}
            params, _ = optimize_func(y, dt, **kwargs)
            _, dydx = diff_func(y, dt, params)
            return dydx
        except Exception as e:
            print(f"{method} derivative fit failed: {e}")
            return None

    for halo_idx in range(n_halos):
        try:
            df_rho = pd.read_csv(os.path.join(density_dir, density_files[i]))
            df_vel = pd.read_csv(os.path.join(velocity_dir, velocity_files[i]))
            df_mass = pd.read_csv(os.path.join(mass_dir, mass_files[i]))
        except Exception as e:
            print(f"[Halo {i}] loading profiles failed: {e}")
            continue

        r = df_rho["r_scaled"].values
        m = df_mass["m_scaled"].values
        rho = df_rho["rho_scaled"].values
        sigma_tot = df_vel["sigma_total_scaled"].values
        sigma_rad = df_vel["sigma_rad_scaled"].values

        dt_r = np.diff(np.log10(r)).mean()
        dt_m = np.diff(np.log10(m)).mean()

        log_rho = np.log10(rho)
        log_sigma_tot = np.log10(sigma_tot)
        log_sigma_rad = np.log10(sigma_rad)

        drho_dlogr = fit_derivative(log_rho, dt_r)
        dsigma_tot_dlogr = fit_derivative(log_sigma_tot, dt_r)
        dsigma_rad_dlogr = fit_derivative(log_sigma_rad, dt_r)

        drho_dlogm = fit_derivative(log_rho, dt_m)
        dsigma_tot_dlogm = fit_derivative(log_sigma_tot, dt_m)
        dsigma_rad_dlogm = fit_derivative(log_sigma_rad, dt_m)

        if any(x is None for x in [drho_dlogr, dsigma_tot_dlogr, dsigma_rad_dlogr,
                                drho_dlogm, dsigma_tot_dlogm, dsigma_rad_dlogm]):
            print(f"[Halo {i}] derivative fitting failed, skipping")
            continue

        slope_Q_tot_r = drho_dlogr - 3 * dsigma_tot_dlogr
        slope_Q_rad_r = drho_dlogr - 3 * dsigma_rad_dlogr
        slope_Q_tot_m = drho_dlogm - 3 * dsigma_tot_dlogm
        slope_Q_rad_m = drho_dlogm - 3 * dsigma_rad_dlogm

        df_r = pd.DataFrame({"r_scaled": r, "slope_Q_r": slope_Q_rad_r, "slope_Q_tot": slope_Q_tot_r})
        df_m = pd.DataFrame({"m_scaled": m, "slope_Q_r": slope_Q_rad_m, "slope_Q_tot": slope_Q_tot_m})

        df_r.to_csv(os.path.join(slope_r_dir, f"slope_profile_r_{halo_idx:03d}.csv"), index=False)
        df_m.to_csv(os.path.join(slope_m_dir, f"slope_profile_m_{halo_idx:03d}.csv"), index=False)


In [ ]:
# NOTE: this is the version that smooth the derivative of derivative of ppsd derictly
def get_diff_and_optimize_funcs(method):
    submodules = [
        'kalman_smooth',
        'smooth_finite_difference',
        'finite_difference',
        'total_variation_regularization',
        'linear_model'
    ]
    for submod in submodules:
        try:
            mod_optimize = getattr(pynumdiff.optimize, submod)
            mod_diff = getattr(pynumdiff, submod)
            if hasattr(mod_optimize, method) and hasattr(mod_diff, method):
                return getattr(mod_diff, method), getattr(mod_optimize, method)
        except AttributeError:
            continue
    raise ValueError(f"Method '{method}' not found in any submodule.")

def fit_and_save_ppsd_slopes(base_dir, suite_name, method='constant_jerk', tvgamma=None):
    ppsd_dir = os.path.join(base_dir, "output", suite_name, "ppsd_profiles")
    slope_r_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_r")
    slope_m_dir = os.path.join(base_dir, "output", suite_name, "ppsd_slope_profiles_m")
    os.makedirs(slope_r_dir, exist_ok=True)
    os.makedirs(slope_m_dir, exist_ok=True)

    ppsd_files = sorted([f for f in os.listdir(ppsd_dir) if f.endswith(".csv")])
    n_halos = len(ppsd_files)

    def fit_derivative(y, dt):
        try:
            diff_func, optimize_func = get_diff_and_optimize_funcs(method)
            kwargs = {'tvgamma': tvgamma} if 'tvgamma' in optimize_func.__code__.co_varnames else {}
            params, _ = optimize_func(y, dt, **kwargs)
            _, dydx = diff_func(y, dt, params)
            return dydx
        except Exception as e:
            print(f"{method} derivative fit failed: {e}")
            return None

    for halo_idx in range(n_halos):
        try:
            df_Q = pd.read_csv(os.path.join(ppsd_dir, ppsd_files[halo_idx]))
        except Exception as e:
            print(f"[Halo {i}] loading profiles failed: {e}")
            continue

        r = df_Q["r_scaled"].values
        m = df_Q["m_scaled"].values
        Q_tot = df_Q["Q_tot"].values
        Q_r = df_Q["Q_r"].values

        dt_r = np.diff(np.log10(r)).mean()
        dt_m = np.diff(np.log10(m)).mean()

        log_Q_r = np.log10(Q_r)
        log_Q_tot = np.log10(Q_tot)

        slope_Q_tot_r = fit_derivative(log_Q_tot, dt_r)
        slope_Q_rad_r = fit_derivative(log_Q_r, dt_r)
        slope_Q_tot_m = fit_derivative(log_Q_tot, dt_m)
        slope_Q_rad_m = fit_derivative(log_Q_r, dt_m)

        df_r = pd.DataFrame({"r_scaled": r, "slope_Q_r": slope_Q_rad_r, "slope_Q_tot": slope_Q_tot_r})
        df_m = pd.DataFrame({"m_scaled": m, "slope_Q_r": slope_Q_rad_m, "slope_Q_tot": slope_Q_tot_m})

        df_r.to_csv(os.path.join(slope_r_dir, f"halo_{halo_idx:03d}_profile.csv"), index=False)
        df_m.to_csv(os.path.join(slope_m_dir, f"halo_{halo_idx:03d}_profile.csv"), index=False)

for suite in suite_names:
    fit_and_save_ppsd_slopes(base_dir, suite, method='constant_jerk', tvgamma=None)